In [ ]:
from progressivis_nb_widgets import *

In [ ]:
"""
Clustering datasets may be found at
http://cs.joensuu.fi/sipu/datasets/
"""
from progressivis import Scheduler, Every#, log_level
from progressivis.cluster import MBKMeans
from progressivis.io import CSVLoader
from progressivis.vis import MCScatterPlot
from progressivis.datasets import get_dataset
from progressivis.stats import RandomTable
from progressivis.core.utils import gather_and_run
from progressivis.utils.psdict import PsDict
import pandas as pd
import numpy as np
from progressivis.datasets.random import generate_random_multivariate_normal_csv as gen_csv
from progressivis.stats.blobs_table import BlobsTable, MVBlobsTable
try:
    s = scheduler
except NameError:
    s = Scheduler()

MULTIVARIATE_NORMAL = True

if not MULTIVARIATE_NORMAL: # simple isotropic blobs (using scikit-learn make_blobs)
    centers = [(0.1, 0.3), (0.7, 0.5), (-0.4, -0.3)]
    data = BlobsTable(columns=['_0', '_1'],  centers=centers, cluster_std=0.2, rows=999999, scheduler=s)
else: # i.e. control each cluster's covariance matrix (using numpy.random.multivariate_normal)
    means = [0.1, 0.3], [0.7, 0.5], [-0.4, -0.3]
    covs = [[0.01, 0], [0, 0.09]], [[0.04, 0], [0, 0.01]], [[0.09, 0.04], [0.04, 0.02]]
    data = MVBlobsTable(columns=['_0', '_1'], means=means, covs=covs, rows=999999, scheduler=s)
mbkmeans = MBKMeans(columns=['_0', '_1'], n_clusters=3, batch_size=100, tol=0.01, is_input=False, scheduler=s)
sp = MCScatterPlot(scheduler=s, classes=[('Scatterplot', '_0', '_1', mbkmeans)])
sp.create_dependent_modules(data,'result')
sp['Scatterplot'].min_value.result = PsDict({'_0': -np.inf, '_1': -np.inf})
sp['Scatterplot'].max_value.result = PsDict({'_0': np.inf, '_1': np.inf})
#mbkmeans.input.table = data.output.table
mbkmeans.create_dependent_modules(sp['Scatterplot'].range_query_2d)
sp.move_point = mbkmeans.moved_center # for input management
# Create the dashboard object

psboard = PsBoard(s)
# Visualisations require registration :
# 1) When widget provides the link_module() method do :
psboard.register_visualisation(Scatterplot(), sp)
# 2) When widget is generic or for overloading the link_module() method do:
#psboard.register_visualisation(FooWidget(), bar_module, glue=glue_func)
# NB: glue_func(widget, module)-> [coroutine] must be provided
# Start application :
gather_and_run(s.start(), *psboard.coroutines)

In [ ]:
#print(psboard.viz_register)
#display(psboard.viz_register['mc_scatter_plot_1'][0][0])
display(psboard)